In [1]:
from transformers import (VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer)
from transformers import BertConfig, ViTConfig, VisionEncoderDecoderConfig
import torch
import numpy as np
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# gpu 사용
decoder_model_name_or_path = 'skt/kogpt2-base-v2'
#decoder_model_name_or_path = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(decoder_model_name_or_path)
# decoder, tokenizer -> gpt2

encoder_model_name_or_path = 'google/vit-base-patch16-224-in21k'

feature_extractor = ViTFeatureExtractor.from_pretrained(encoder_model_name_or_path)
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder_model_name_or_path, decoder_model_name_or_path)
# encoder, extractor -> vit
model.to(device)
print("Model loaded")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['transformer.h.7.crossattention.q_attn.weight', 'transformer.h.11.crossattention.c_proj.bias', 'transformer.h.5.crossattention.masked_bias', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.2.crossattention.bias', 'transformer.h.11.crossattention.masked_bias', 'transformer.h.4.ln_cross_attn.weight', 'transformer.h.4.crossattention.c_proj.bias', 'transformer.h.4.crossattention.masked_bias', 'transformer.h.5.crossattention.c_attn.weight', 'transformer.h.3.crossattention.c_attn.weight', 'transformer.h.5.crossattention.bias', 'transformer.h.1.crossattention.bias', 'transformer.h.8.crossattention.q_attn.weight', 'transformer.h.2.ln_cross_attn.weight', 'transformer.h.2.crossattention.c_proj.weight', 'transformer.h.3.crossattention.c_proj.weigh

Model loaded


In [4]:
image = Image.open("val2014/COCO_val2014_000000391895.jpg").convert("RGB")
image_tensor = np.array(image)
image_tensor.shape

(360, 640, 3)

In [32]:
#image
model.config.decoder_start_token_id = 0
model.config.pad_token_id = 3
# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [26]:
tokenizer.bos_token_id

51200

In [33]:
text_list = [
    "빨간 헬멧을 쓴 남자가 작은 오토바이를 타고 비포장 도로에 있다.", 
    "시골의 비포장 도로에서 오토바이를 타는 남자", 
    "오토바이 뒤에 탄 남자", 
    "오토바이 위에 젊은이가 탄 비포장 도로는 다리가 있는 초록빛 지역의 전경과 구름 낀 산의 배경이 있다.", 
    "빨간 셔츠와 빨간 모자를 쓴 남자가 언덕 쪽 오토바이 위에 있다."
]

In [34]:
pixel_values = feature_extractor(image_tensor, return_tensors="pt").pixel_values
#pixel_values = feature_extractor(image_tensor, return_tensors="pt")
print(pixel_values.shape)
#print(pixel_values)

torch.Size([1, 3, 224, 224])


In [35]:
text = text_list[0]
labels = tokenizer(text, return_tensors="pt").input_ids

In [36]:
print(labels)
print(tokenizer.decode(labels[0]))
print(tokenizer.vocab_size)

tensor([[22162, 14381,  7527,  8137, 10097, 28736,  9836, 19218,  7596,  9989,
         12814,  9072, 31880, 10794,  8022, 10960]])
빨간 헬멧을 쓴 남자가 작은 오토바이를 타고 비포장 도로에 있다.
51200


In [37]:
#outputs = model(pixel_values=pixel_values.to(device))
outputs = model(pixel_values=pixel_values.to(device), labels=labels.to(device))

In [38]:
loss = outputs.loss

In [39]:
generated_ids = model.generate(pixel_values.to(device))
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [40]:
generated_text

"기 때문에 그걸로 끝내는 게 더 낫다.\n그런데 이게 바로 '수"